In [1]:
%gui qt
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point,LineString,  mapping


In [2]:
import json
import geojson
import pathlib


## Make a napari viewer...

In [3]:
v = napari.Viewer()

## Load Cell table from 1 dataset

In [4]:
# downloaded Baysor-segmented data for Baristaseq
cell_stats = pd.read_csv("/Users/brian/Downloads/mouse_visp1_ISS_m_brain_03_PreliminaryCalls_prior/segmentation_cell_stats.csv")


In [5]:
# add the cells to the napari viewer
v.add_points(cell_stats[["x","y"]].values)

<Points layer 'Points' at 0x13055dd90>

In [6]:
# Brian's layer segmentation and restriction of cells to a rectangular region that includes roughly equal area in each layer



with open("/Users/brian/work/data/data/annotations/ISS_layer_Annotation_new_6_layers.json", 'r') as r:
    layer_geojson = geojson.load(r)

In [8]:
layer_geojson

[{"coordinates": [[[39334.13312, 1362.231682], [39035.589057, 645.725932], [39354.036057, 386.987744], [41384.135683, 247.667182], [45703.073121, 227.764244], [49325.407747, 904.464119], [53962.792186, 2874.854932], [57366.194499, 5163.692745], [60072.994, 8507.386246], [61784.646626, 11453.020997], [63038.531688, 14657.393935], [62361.831813, 15334.09381], [61685.131938, 13184.57656], [59774.449938, 9323.406684], [57346.291562, 6278.257246], [54739.006749, 4287.963495], [51514.730873, 2536.504995], [49126.378372, 1760.290432], [45683.170184, 824.852369], [41483.65037, 944.269994]]], "name": "L1_polygon", "type": "Polygon"},
 {"coordinates": [[[39393.841932, 1402.037557], [39732.19187, 1899.610994], [44946.761496, 1800.096307], [48171.037372, 2357.378557], [51833.177873, 3949.613558], [54818.618499, 5382.625058], [57226.873937, 7552.045246], [59575.420562, 10238.941809], [61764.743688, 15791.861373], [62262.317126, 15493.31731], [61545.811375, 13184.57656], [59674.93525, 9403.018434], 

In [10]:
    
layer_data_brl = {layer_annotation["name"]:np.array(layer_annotation["coordinates"][0]) 
                  for layer_annotation in layer_geojson if layer_annotation["name"] not in ["column_axis"]}

In [11]:
# add the polygons to the viewer
v.add_shapes([layer_data_brl[layer_name] for layer_name in layer_data_brl.keys()],shape_type='polygon', edge_width=1,
    edge_color='coral',
    face_color='royalblue',
    name="Layers")

<Shapes layer 'Layers' at 0x137362750>

## Brian added a "VISp" polygon to subset the cortical layer annotations from Christoffer to include only V1 and merged the previously separate layer 2 and 3 annotations

In [27]:
shapes_layer  = v.layers[-1].data
annotation_names = ["VISp_1", "VISp_2/3", "VISp_4", "VISp_5","VISp_6", "VISp"]
annotations = [mapping(Polygon(p))  for ii,p in enumerate(shapes_layer)]
for ii,ai in enumerate(annotations):
    ai["name"] = annotation_names[ii]

a = geojson.GeometryCollection(annotations)

with open("/Users/brian/Downloads/mouse_visp1_ISS_m_brain_03_PreliminaryCalls_prior/Layers_VISp_geo.json",'w') as w:
    geojson.dump(a,w)
    

In [46]:
annotations

[{'type': 'Polygon',
  'coordinates': (((39334.13312, 1362.231682),
    (39035.589057, 645.725932),
    (39354.036057, 386.987744),
    (41384.135683, 247.667182),
    (45703.073121, 227.764244),
    (49325.407747, 904.464119),
    (53962.792186, 2874.854932),
    (57366.194499, 5163.692745),
    (60072.994, 8507.386246),
    (61784.646626, 11453.020997),
    (63038.531688, 14657.393935),
    (62256.22292006615, 15439.08804782239),
    (61545.58751585353, 13172.73729925241),
    (59740.189461907954, 9427.496655429137),
    (57377.80690195789, 6469.716864922553),
    (54631.29709648749, 4337.81065228469),
    (51500.660045496756, 2590.031685167163),
    (49138.277485546685, 1783.3644695744579),
    (45682.95460899599, 1033.8837465883028),
    (41478.58297014514, 1117.2565552907276),
    (39334.13312, 1362.231682)),),
  'name': 'VISp_1'},
 {'type': 'Polygon',
  'coordinates': (((39323.02357149946, 1360.5180814896873),
    (40084.29447120469, 2749.3231012221863),
    (43592.31307660307, 2

In [69]:
# Use layers to annotate the cell x gene data
# slightly awkward dict comprehension here going through the annotations (except not annotations[-1])
# and figuring out if the cell location is in the layer polygon and VISp (annotations[-1])
is_in_layer = {p["name"]:[(Polygon(p["coordinates"][0]).intersects(Point(a)) and Polygon(annotations[-1]["coordinates"][0]).intersects(Point(a))) for a  in cell_stats[["x","y"]].values] for ii,p in enumerate(annotations[:-1])}

In [70]:


cell_stats["layer"] = "outside_VISp"
for k in is_in_layer.keys():
    cell_stats.loc[is_in_layer[k],["layer"]] = k


In [75]:
cell_stats["x_um"] = cell_stats["x"]*0.1625
cell_stats["y_um"] = cell_stats["y"]*0.1625

In [76]:
cell_stats

,cell,x,y,has_center,cluster,n_transcripts,density,elongation,area,avg_confidence,doublet_score,layer,x_um,y_um
0,1,152.142857,24494.571429,False,3,7,0.04590,2.794,152.5,0.9991,0.284900,outside_VISp,24.723214,3980.367857
1,2,128.500000,26559.750000,False,3,4,0.05714,3.045,70.0,0.9987,NaN,outside_VISp,20.881250,4315.959375
2,3,130.833333,26721.833333,False,3,6,0.02353,2.440,255.0,0.9881,0.003106,outside_VISp,21.260417,4342.297917
3,4,193.428571,24838.857143,False,3,7,0.02708,9.700,258.5,0.9961,0.003106,outside_VISp,31.432143,4036.314286
4,5,214.000000,24864.600000,False,3,5,0.03953,2.221,126.5,0.9975,0.137000,outside_VISp,34.775000,4040.497500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135207,135208,46897.000000,34441.000000,False,2,1,NaN,NaN,NaN,0.9997,NaN,outside_VISp,7620.762500,5596.662500
135208,135209,20717.000000,6161.000000,False,1,2,NaN,NaN,NaN,0.9993,NaN,outside_VISp,3366.512500,1001.162500
135209,135210,46620.000000,7199.500000,False,2,2,NaN,NaN,NaN,0.9995,NaN,VISp_6,7575.750000,1169.918750
135210,135211,32742.000000,3575.000000,False,1,1,NaN,NaN,NaN,0.9989,NaN,outside_VISp,5320.575000,580.937500


In [77]:
cell_stats.to_csv("/Users/brian/Downloads/mouse_visp1_ISS_m_brain_03_PreliminaryCalls_prior/segmentation_cell_stats_BRL_layers.csv")

In [72]:
v.add_points(cell_stats.loc[cell_stats["layer"]=="VISp_5"][["x","y"]].values)

<Points layer 'Points [3]' at 0x1308cc550>

In [73]:
cell_stats["layer"].unique()

array(['outside_VISp', 'VISp_1', 'VISp_2/3', 'VISp_4', 'VISp_5', 'VISp_6'],
      dtype=object)